In [53]:
import random


def split_dataset(x: int, k: int):
    end = False
    step = 0
    start = random.randint(0, x - 1)
    subsets = []
    l = list(range(x))
    l_bool = [False for elem in l]

    while not end:
        # if there are enough indexes to make up another subset, make it
        if l_bool.count(False) >= int(len(l) / k):
            l_sub = []
            while len(l_sub) < int(len(l) / k):
                idx = (start + step) % len(l)
                l_sub.append(l[idx])
                l_bool[idx] = True
                step += 1
            subsets.append(l_sub)
        else:
            # if there are still indexes that haven't been assigned
            l_sub_last = []
            while l_bool.count(False) > 0:
                idx = (start + step) % len(l)
                l_sub_last.append(l[idx])
                l_bool[idx] = True
                step += 1
            subsets.append(l_sub_last)
            if len(l_sub_last) < 100:
                # if the last sublist is too short merge it with the last one
                merged_sub = subsets[-2] + subsets[-1]
                subsets.pop(-1)
                subsets.pop(-2)
                subsets.append(merged_sub)
            end = True
    return subsets

In [54]:
split_dataset(33, 5)

[[12, 13, 14, 15, 16, 17],
 [18, 19, 20, 21, 22, 23],
 [24, 25, 26, 27, 28, 29],
 [3, 4, 5, 6, 7, 8],
 [3, 4, 5, 6, 7, 8, 9, 10, 11]]

In [55]:
import numpy as np
from typing import List

def split_dataset_indices(n_data: int, k: int) -> List[List[int]]:
    # randomly roll indices
    start = np.random.randint(0, n_data - 1)
    idxs = list(np.roll(list(range(n_data)), start))

    # get the len of each subset
    sub_len, extras = divmod(n_data, k)
    subsets_lens = [sub_len + 1 if i < extras else sub_len for i in range(k)]

    # get start and end indices of each "subset of indices"
    cum_lens = np.cumsum([0] + subsets_lens)
    return [idxs[s:e] for s, e in zip(cum_lens[:-1], cum_lens[1:])]

In [56]:
split_dataset_indices(33, 5)

[[6, 7, 8, 9, 10, 11, 12],
 [13, 14, 15, 16, 17, 18, 19],
 [20, 21, 22, 23, 24, 25, 26],
 [27, 28, 29, 30, 31, 32],
 [0, 1, 2, 3, 4, 5]]